# 🧠 Módulo 5 — Programación Funcional en Python

Python no es 100% funcional, pero soporta muchas herramientas del paradigma:

- Funciones como ciudadanos de primera clase
- Funciones puras
- Lambdas
- `map`, `filter`, `reduce`
- Higher-order functions
- Closures

Este notebook explica cómo usarlas de forma realista en Python moderno.

---
## 1️⃣ Funciones como "ciudadanos de primera clase"

En Python, las funciones:
- se pueden almacenar en variables
- pasar como parámetros
- devolver desde otras funciones
- guardarse en estructuras


In [ ]:
def saludar():
    return "Hola"

f = saludar
f(), f()

---
## 2️⃣ Funciones puras

Una **función pura**:
- Siempre produce el mismo resultado con los mismos parámetros
    - Sin efectos secundarios
    - No modifica variables externas


In [ ]:
def pura(x, y):
    return x + y

pura(2,3), pura(2,3)

Una función **no pura** afecta o depende del estado externo:

In [ ]:
contador = 0
def impura(x):
    global contador
    contador += x
    return contador

impura(1), impura(1)

---
## 3️⃣ Lambdas

Funciones pequeñas y anónimas:

In [ ]:
cuadrado = lambda x,y: x*y
 
# def cuadrado(x,y):
#     return x*y

cuadrado(7,5)


# serieFactor = lambda x: [i*x for i in range(1,11)]
# serieExponente = lambda x: ([i**x for i in range(1,11)] , [i**x for i in range(1,11)])

# serieExponente(2)

---
## 4️⃣ `map`, `filter`, `reduce`

### ✔️ map: aplicar transformación

In [ ]:
inc = lambda x: x+1
dec = lambda x: x-1
strl = lambda x: str(x)

nums = [1,2,3,4]
list(map(lambda x: str(x), nums))


### ✔️ filter: filtrar valores

In [ ]:
par = lambda x: x%2==0
# impar = lambda x: x%2!=0

def impar(x):
    return x%2!=0

list(filter(lambda x: x%2!=0, nums))

class Ejemplo():
    soypropiedad = lambda self,x:x

e = Ejemplo()

e.soypropiedad(1)

### ✔️ reduce: acumular resultados
Necesita import explícito:

In [ ]:
from functools import reduce

# pares = [("id", 1), ("name", "Ana"), ("age", 30)]

# d = reduce(lambda acc, x, i, iter: 
#        (
#            acc.update({x[0]:x[1]}) 
#            or acc
#        )
#        , pares, {})

# dict(d)

class Pedido():
    pass

nums = [1,2,3,4,5,6,7]

reduce(lambda a, c: a+str(c), nums, '')

# sum(nums)


---
## 5️⃣ Higher-Order Functions (HOF)

Una **HOF** es una función que:
- recibe otra función como argumento
- o devuelve una función nueva

Ejemplo: creador de multiplicadores:

In [ ]:
def multiplicador(n):
    def multiplicar(x):
        return x*n
    return multiplicar

por_3 = multiplicador(3)
por_4 = multiplicador(4)


por_4(10),por_3(10)

In [ ]:
def log(fn):
    def wrapper(*args, **kvargs):
        print(f"[LOG] {fn.__name__}")
        return fn(*args, **kvargs)
    return wrapper

from functools import wraps

def cache(**props):
    def decorator(fn):
        memo = {}
        @wraps(fn)
        def wrapper(*args, **kwargs):
            key = (args, frozenset(kwargs.items()))
            if key not in memo:
                print("⏳ Ejecutando la función…")
                memo[key] = fn(*args, **kwargs)
            else:
                print("⚡ HIT en caché:", key)
            return memo[key]
        return wrapper 
    return decorator



# @log
# def procesar():
#     print("procesando..")

# @log
# def otrafuncion():
#     print("otrafuncion")

@cache(timeout=1000)
def calcular(a,b):
    print('calculando')
    return a+b




In [ ]:
import time

def timer(fn):
    @wraps(fn)
    def wrapper(*args, **kvargs):
        ini = time.perf_counter()
        res = fn(*args, **kvargs)
        fin = time.perf_counter()
        print(f"{fn.__name__} ejecutada en {(fin - ini) * 1000:3f} ms")
        return res
    return wrapper

In [ ]:
@timer
def calcular_lento():
    suma = 0
    for _ in range(10_000_000):
        suma += 1
    return suma

In [ ]:
calcular_lento()

In [ ]:
calcular(3,3)

In [ ]:
def pipeline(data, *steps):
    for step in steps:
        data = map(step, data)
    return list(data)



r = pipeline([1,2,3], 
         lambda x:x*2,
         lambda x:x+1
         )
r

---
## 6️⃣ Closures

Una función interna recuerda las variables de su función externa, incluso después de haber terminado.

In [ ]:
def contador():
    n = 0
    def inc():
        nonlocal n
        n += 1
        return n
    return inc

c = contador()
o = contador()
c(), c(), c()
o()
c()

In [ ]:
c()

In [ ]:
# CONTROL DE ACCESO

def require_role(role):
    def decorator(fn):
        def wrapper(user, *args):
            if user['role'] != role:
                raise PermissionError('Acceso denegado')
            return fn(user, *args)
        return wrapper
    return decorator


@require_role("admin")
def borrar(user):
    return " usuario borrado "





In [ ]:
def aplicar_estrategia(f):
    def procesar(x):
        return f(x)
    return procesar

doble = aplicar_estrategia(lambda v:v*2)
triple = aplicar_estrategia(lambda v:v*3)

doble(2), triple(2)





In [7]:
def usuario(nombre, edad):
        _edad = edad
        def incrementar():
            nonlocal _edad
            _edad +=1
            return -edad
        def decrementar():
            nonlocal _edad
            _edad -=1
            return -edad
        def creausario():
             return {
                  "nombre": nombre,
                  "get_edad": lambda: _edad,
                  "incrementar": incrementar,
                  "decrementar": decrementar
             }
    
        return creausario()



u = usuario('david',30)

In [9]:
print(u["nombre"])          # david
print(u["get_edad"]())      # 30
print(u["incrementar"]())   # 31
print(u["decrementar"]())   # 30
print(u["get_edad"]())      # 30


david
30
-30
-30
30


---
## 7️⃣ Ejemplo real: pipeline funcional

Pipeline para limpieza de datos:

In [10]:
datos = ["  Hola  ", "MUNDO", "  Python "]
resultado = list(
    map(lambda t: t.strip().lower(), datos)
)
resultado

['hola', 'mundo', 'python']

In [11]:
# VALIDADOR

def validar(regla):
    def check(x):
        if not regla(x):
            raise ValueError('Dato Invalido')
        return x
    return check

# Mis reglas

solo_positivos = validar(lambda x: x > 0)


In [44]:

from functools import wraps

def validar_args(regla):
    def decorador(fn):
        @wraps(fn)
        def wrapper(*args, **kvargs):
            nargs = [validar(regla)(x) for x in args]
            nkvargs = {k: validar(regla)(v) for k,v in kvargs.items()}
            return fn(*nargs,**nkvargs)
        return wrapper
    return decorador

In [47]:

@validar_args(lambda x:x>0)
def multiplicar(a, b):
    return a * b

@validar_args(lambda x: isinstance(x,str) & x != '')
def concatenar(a,b):
    return a + b


In [48]:
r = multiplicar(2,1)
r

r = concatenar('Hola','')
r


TypeError: unsupported operand type(s) for &: 'bool' and 'str'

---
## 8️⃣ Ejercicio práctico

### �� Ejercicio
Dado:
```python
numeros = [1,2,3,4,5,6,7,8,9,10]
```
Usa **solo funciones funcionales** (`map`, `filter`, `reduce`) para obtener:

1. Los números pares
2. Su cuadrado
3. La suma final

Escribe tu solución aquí:

In [60]:
# Escribe tu solución aquí
from functools import reduce

numeros = [1,2,3,4,5,6,7,8,9,10]

pares = list(filter(lambda x:x%2==0 ,numeros))
cuadrados = list(map(lambda x: x*x, pares))
suma = reduce(lambda a,c: a+c ,cuadrados,0)

pares,cuadrado, suma

([2, 4, 6, 8, 10], [4, 16, 36, 64, 100], 220)

---
## ✅ Solución (oculta)

<details>
<summary>Mostrar solución</summary>

```python
from functools import reduce

pares = list(filter(lambda x: x%2==0, numeros))
cuadrados = list(map(lambda x: x*x, pares))
total = reduce(lambda acc,x: acc+x, cuadrados)

pares, cuadrados, total
```
</details>